Plotagem Grafo - Entrega 1

In [ ]:
import netpixi
import graph_tool_extras as gte
import pandas as pd
import csv
from graph_tool import draw

In [ ]:
PATH = 'relacao_comercial.csv'

# Exemplo: caso a coluna com o valor esteja nomeada como "TradeValue in 1000 USD"
col_valor = 'TradeValue in 1000 USD'

In [ ]:
df = pd.read_csv(PATH)

In [ ]:
df.head()

In [ ]:
# Filtrar as linhas para excluir as que têm "Unspecified" ou "World" nas colunas "ReporterName" ou "PartnerName"
df = df[~df['ReporterName'].isin(['Unspecified', 'World']) & ~df['PartnerName'].isin(['Unspecified', 'World'])]

In [ ]:
thresholds_por_ano = df.groupby('Year')[col_valor].quantile(0.3)

In [ ]:
# 3. Criar o dicionário com os thresholds por ano
thresholds_dict = thresholds_por_ano.to_dict()

# 4. Inicializar uma lista para armazenar as estatísticas
estatisticas_por_ano = []

In [ ]:
# 5. Filtrar o DataFrame usando o threshold de cada ano
#    Vamos criar uma lista de DataFrames filtrados e depois concatená-los
dfs_filtrados = []
for ano, limite in thresholds_por_ano.items():
    # Contar o número total de relações antes do filtro
    total_relacoes_ano = len(df[df['Year'] == ano])

    # Seleciona apenas as linhas do ano correspondente cujo valor esteja acima do threshold
    filtro_ano = df[(df['Year'] == ano) & (df[col_valor] >= limite)]
    dfs_filtrados.append(filtro_ano)

    # Contar o número de relações cortadas e mantidas
    cortadas = total_relacoes_ano - len(filtro_ano)
    mantidas = len(filtro_ano)

    # Armazenar as estatísticas
    estatisticas_por_ano.append({
        'Ano': ano,
        'Total': total_relacoes_ano,
        'Cortadas': cortadas,
        'Mantidas': mantidas
    })

In [ ]:
# 6. Concatenar o resultado em um único DataFrame filtrado
df_filtrado = pd.concat(dfs_filtrados, ignore_index=True)

# 7. Exibir as estatísticas
estatisticas_df = pd.DataFrame(estatisticas_por_ano)

In [ ]:
print("Thresholds por ano:")
print(thresholds_por_ano)

In [ ]:
print("\nEstatísticas por ano:")
print(estatisticas_df)

In [ ]:
df_filtrado.head()

In [ ]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
        u['saldo'] = 0
    return u

def get_or_add_edge(g, exporter, importer):
    e = g.edge_by_ids(exporter, importer)
    if e is None:
        e = g.add_edge_by_ids(exporter, importer)
        e['total'] = 0
    return e

In [ ]:
graphs_by_year = {}

with open(PATH, encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # Ignorar cabeçalho

    for line in reader:
        reporter_iso3 = line[0]  # País exportador
        partner_iso3 = line[2]  # País importador
        year = int(line[4])  # Ano de referência
        
        # Verifica se a coluna de valor comercial está vazia ou inválida
        if line[6].strip() == '':
            continue  # Pula esta linha

        try:
            trade_value = float(line[6])  # Valor comercial (em milhares de USD)
        except ValueError:
            continue

        # Verifica o valor mínimo para o ano a partir de thresholds_dict
        if year in thresholds_dict:
            VALOR_MINIMO = thresholds_dict[year]  # Ajuste para o valor mínimo do ano especifico

            # Filtrar relações comerciais pelo valor mínimo
            if trade_value >= VALOR_MINIMO:
                # Criar grafo para o ano, se ainda não existir
                if year not in graphs_by_year:
                    g = gte.Graph(directed=True)
                    g.add_vp("saldo")
                    g.add_ep("total")
                    g.add_gp("ano")
                    g["ano"] = year
                    graphs_by_year[year] = g

                g = graphs_by_year[year]

                # Criar ou obter os vértices dos países
                u = get_or_add_vertex(g, reporter_iso3)
                v = get_or_add_vertex(g, partner_iso3)

                # Criar ou atualizar a aresta representando a relação comercial
                e = get_or_add_edge(g, reporter_iso3, partner_iso3)
                e["total"] += trade_value




In [ ]:
YEAR_TO_PLOT = 2020

if YEAR_TO_PLOT in graphs_by_year:
    gte.save(graphs_by_year[YEAR_TO_PLOT], "primeira-versao.net.gz")
else:
    print(f"Nenhum dado disponível para o ano {YEAR_TO_PLOT}.")

In [ ]:
h = gte.load("primeira-versao.net.gz")
layout = draw.sfdp_layout(h)
gte.move(h, layout)
h = gte.clean(h)
gte.save(h, 'relacao-comercial.net.gz')
r = netpixi.render('relacao-comercial.net.gz')

In [ ]:
r.vertex_default(
    size=10,
    color=0x3b3fd9,
    bwidth=0.5,
    bcolor=0x000487,
)

r.edge_default(
    width=0.5,
    color=0xd7d7de,
    curve1=0.2,
    curve2=0.2,
)

In [ ]:
# Exibe a quantidade de vértices e arestas no grafo selecionado
num_vertices = h.num_vertices()
num_arestas = h.num_edges()

print(f'O grafo do ano {YEAR_TO_PLOT} contém {num_vertices} vértices e {num_arestas} arestas.')

Visualização em que o tamanho dos vértices corresponde ao DEGREE.

In [ ]:
g = gte.load('relacao-comercial.net.gz')

c = gte.total_degree(g)
g.add_vp('degree', c)

gte.save(g, 'degree.net.gz')
r = netpixi.render('degree.net.gz')

In [ ]:
r.vertex_scale('degree', 5, 20)
r.vertex_default(
    size=10,
    color=0x3b3fd9,
    bwidth=0.5,
    bcolor=0x01011f,
)

r.edge_default(
    width=0.5,
    color=0xd7d7de,
    curve1=0.2,
    curve2=0.2,
)

Visualização em que o tamanho dos vértices corresponde ao BETWEENNESS.

In [ ]:
from graph_tool import centrality

In [ ]:
g = gte.load('relacao-comercial.net.gz')

c, _ = centrality.betweenness(g)

g.add_vp('betweenness', c)

gte.save(g, 'betweenness.net.gz')

r = netpixi.render('betweenness.net.gz')

In [ ]:
r.vertex_scale('betweenness', 5, 20)
r.vertex_default(
    size=10,
    color=0x3b3fd9,
    bwidth=0.5,
    bcolor=0x01011f,
)

r.edge_default(
    width=0.5,
    color=0xd7d7de,
    curve1=0.2,
    curve2=0.2,
)

Visualização em que o tamanho dos vértices corresponde ao CLOSENESS

In [ ]:
g = gte.load('relacao-comercial.net.gz')

# O parâmetro harmonic=True é necessário para garantir
# que a fórmula seja exatamente a definida acima.
c = centrality.closeness(g, harmonic=True)

g.add_vp('closeness', c)

In [ ]:
gte.save(g, 'closeness.net.gz')

In [ ]:
r = netpixi.render('closeness.net.gz')

In [ ]:
r.vertex_scale('closeness', 5, 20)

Visualização em que o tamanho dos vértices correspondem à LACUNAS ESTRUTURAIS

In [ ]:
g = gte.load('../../../Datasets/florentine-families-clean.net.gz')

c = gte.burt_constraint(g)

g.add_vp('burt_constraint', c)

In [ ]:
gte.save(g, 'structural_holes.net.gz')

In [ ]:
r = netpixi.render('structural_holes.net.gz')

In [ ]:
r.vertex_scale('burt_constraint', 5, 20)

Código utilizado para encontrar os 5 principais nós da rede, levando em conta o DEGREE de cada um deles.

In [ ]:
import graph_tool.all as gt
import graph_tool_extras as gte

# Carrega o grafo
g = gte.load('teste-reposicionado (1).net.gz')

# Calcula o grau total
degree_prop = g.degree_property_map("total")
g.vertex_properties['degree'] = degree_prop

# Acessa a propriedade 'id', que representa o nome do vértice
id_prop = g.vertex_properties['id']

# Ordena os vértices pelo grau (do maior para o menor)
sorted_vertices = sorted(g.vertices(), key=lambda v: degree_prop[v], reverse=True)

# Número de vértices que você quer mostrar
top_n = 5

# Mostra os top_n vértices com maior grau
print(f"Top {top_n} nós com maior grau:")
for v in sorted_vertices[:top_n]:
    print(f"{id_prop[v]} — Grau: {degree_prop[v]}")